Importing the necessary libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dropout,Flatten,Dense,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import os

Applying Data Augmentation 

In [ ]:
datagen = ImageDataGenerator(
rotation_range=15,
rescale=1/255,
zoom_range=0.1,
horizontal_flip=True,width_shift_range=0.1,height_shift_range=0.1, validation_split=0.2)


Preparing training data

In [ ]:
trainDatagen = datagen.flow_from_directory(directory='../input/cell-images-for-detecting-malaria/cell_images/cell_images/',
                                           target_size=(124, 124),
                                           class_mode = 'binary',
                                           batch_size = 64,
                                           subset='training')

Assigning each value a name

In [ ]:
trainDatagen.class_indices

Preparing validation data

In [ ]:
valDatagen = datagen.flow_from_directory(directory='../input/cell-images-for-detecting-malaria/cell_images/cell_images/',
                                           target_size=(124, 124),
                                           class_mode = 'binary',
                                           batch_size = 64,
                                           subset='validation')

In [ ]:
Adding layers to the CNN

In [ ]:
model = Sequential()
model.add(Conv2D(16,(3,3),activation='relu',input_shape=(124, 124, 3)))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.25))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

model.summary()


Import callback functions

In [ ]:
from keras.callbacks import EarlyStopping,ReduceLROnPlateau

Stop training when a monitored metric has stopped improving.

In [ ]:
earlystop=EarlyStopping(patience=10)

Decrease learning rate if model does not perform  with good accuracy 

In [ ]:
learning_rate_reduce=ReduceLROnPlateau(monitor='val_acc',min_lr=0.00001)

Store both callbacks

In [ ]:
callbacks=[earlystop,learning_rate_reduce]

Start training the model with 10 epochs

In [ ]:
history=model.fit_generator(generator=trainDatagen,
                            steps_per_epoch=len(trainDatagen),
                            epochs=10,
                            validation_data=valDatagen ,
                            validation_steps=len(valDatagen )
                           )

Plot the accuracy and loss

In [ ]:
plt.plot(history.history['val_loss'],color='r')
plt.plot(history.history['accuracy'],color='black')
plt.plot(history.history['loss'],color='black')
plt.plot(history.history['val_accuracy'],color='yellow')
plt.show() 

Save the model in a .h5 format

In [ ]:
model.save('malaria.h5')